In [46]:
import numpy as np
from sklearn.cross_validation import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.svm import SVR
from sklearn.grid_search import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import xgboost as xgb

In [21]:
# Load Train dataset
data = np.loadtxt('train.in')
# helpfulness | 7| 8| 9| 10| 11| 12-31| 32| 33| 34
help_data_X = data[:, 7:34]
help_data_y = data[:, 34]

# clarity | 3| 4| 5| 6| 9| 10| 11| 12-31|32 |33| 35
clar_data_X1 = data[:, 3:7]
clar_data_X2 = data[:, 9:34]
clar_data_X = np.concatenate((clar_data_X1, clar_data_X2), axis=1)
clar_data_y = data[:, 35]

quality = data[:, 36]

In [22]:
hX_train, hX_test, hy_train, hy_test = train_test_split(help_data_X, help_data_y, test_size=0.3, random_state=42)

In [23]:
cX_train, cX_test, cy_train, cy_test = train_test_split(clar_data_X, clar_data_y, test_size=0.3, random_state=42)

In [50]:
# svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
#                                "gamma": np.logspace(-2, 2, 5)})

# svr = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0],
#                                "gamma": np.logspace(-2,5)})
# svr.fit(hX_train, hy_train)
# y_svr = svr.predict(hX_test)
# lr = LogisticRegression()
# lr = AdaBoostClassifier(n_estimators=400,learning_rate=0.5)
lr = xgb.XGBClassifier(n_estimators=400,max_depth=5)
lr.fit(hX_train, hy_train)
y_lr = lr.predict(hX_test)
mse = mean_squared_error(hy_test, y_lr)
print("MSE HELP: {}".format(mse))

MSE HELP: 2.36566319602


In [51]:
# svr_c = GridSearchCV(SVR(kernel='rbf', gamma=0.1), cv=5,
#                    param_grid={"C": [1e0, 1e1, 1e2, 1e3],
#                                "gamma": np.logspace(-2, 2, 5)})
# svr_c.fit(cX_train, cy_train)
# y_svr_c = svr_c.predict(cX_test)
# mse = mean_squared_error(cy_test, y_svr_c)
# print("MSE: {}".format(mse))
# clr = LogisticRegression()
clr = AdaBoostClassifier(n_estimators=500,learning_rate=0.3)
# clr = xgb.XGBClassifier(n_estimators=500,max_depth=5)
clr.fit(cX_train, cy_train)
y_clr = clr.predict(cX_test)
mse = mean_squared_error(cy_test, y_clr)
print("MSE CLAR: {}".format(mse))

MSE CLAR: 2.0418741512


In [34]:
# y_quality = [0 for i in range(len(y_clr))]
# for i in range(len(y_clr)):
#     y_quality[i] = y_clr[i] + y_lr[i]
# mse = mean_squared_error(quality, y_quality)
# print("MSE ALL: {}".format(mse))
    


In [52]:
# Load Test dataset
data = np.loadtxt('test.in')
# helpfulness | 7| 8| 9| 10| 11| 12-31| 32| 33| 34
help_test = data[:, 7:34]

# clarity | 3| 4| 5| 6| 9| 10| 11| 12-31|32 |33| 35
clar_test1 = data[:, 3:7]
clar_test2 = data[:, 9:34]
clar_test = np.concatenate((clar_test1, clar_test2), axis=1)


In [53]:
# lr = LogisticRegression()
# lr = AdaBoostClassifier(n_estimators=400,learning_rate=0.5)
lr = xgb.XGBClassifier(n_estimators=400,max_depth=5)
lr.fit(help_data_X, help_data_y)
y_lr = lr.predict(help_test)
# clr = LogisticRegression()
clr = AdaBoostClassifier(n_estimators=500,learning_rate=0.3)
clr.fit(clar_data_X, clar_data_y)
y_clr = clr.predict(clar_test)
y_quality = [0 for i in range(len(y_clr))]
outfile = open('predit.out','wb')
for i in range(len(y_clr)):
    y_quality[i] = y_clr[i] + y_lr[i]
    outfile.write(str(y_quality[i]) + '\n')

In [54]:
outfile.close()